In [1]:
import pandas as pd
import numpy as np
import datetime 

import matplotlib
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
%matplotlib inline

In [2]:
import scipy.stats
from IPython import display
from ipywidgets import interact, widgets

import re
import mailbox
import csv
import seaborn as sns
from pandas import Series, DataFrame

In [3]:
movie_info = pd.read_csv('data/rt.movie_info.tsv', sep="\t")
movie_info.head(10)

,id,synopsis,rating,genre,director,writer,theater_date,dvd_date,currency,box_office,runtime,studio
0,1,"This gritty, fast-paced, and innovative police...",R,Action and Adventure|Classics|Drama,William Friedkin,Ernest Tidyman,"Oct 9, 1971","Sep 25, 2001",NaN,NaN,104 minutes,NaN
1,3,"New York City, not-too-distant-future: Eric Pa...",R,Drama|Science Fiction and Fantasy,David Cronenberg,David Cronenberg|Don DeLillo,"Aug 17, 2012","Jan 1, 2013",$,"600,000",108 minutes,Entertainment One
2,5,Illeana Douglas delivers a superb performance ...,R,Drama|Musical and Performing Arts,Allison Anders,Allison Anders,"Sep 13, 1996","Apr 18, 2000",NaN,NaN,116 minutes,NaN
3,6,Michael Douglas runs afoul of a treacherous su...,R,Drama|Mystery and Suspense,Barry Levinson,Paul Attanasio|Michael Crichton,"Dec 9, 1994","Aug 27, 1997",NaN,NaN,128 minutes,NaN
4,7,NaN,NR,Drama|Romance,Rodney Bennett,Giles Cooper,NaN,NaN,NaN,NaN,200 minutes,NaN
5,8,The year is 1942. As the Allies unite overseas...,PG,Drama|Kids and Family,Jay Russell,Gail Gilchriest,"Mar 3, 2000","Jul 11, 2000",NaN,NaN,95 minutes,Warner Bros. Pictures
6,10,Some cast and crew from NBC's highly acclaimed...,PG-13,Comedy,Jake Kasdan,Mike White,"Jan 11, 2002","Jun 18, 2002",$,"41,032,915",82 minutes,Paramount Pictures
7,13,"Stewart Kane, an Irishman living in the Austra...",R,Drama,Ray Lawrence,Raymond Carver|Beatrix Christian,"Apr 27, 2006","Oct 2, 2007",$,"224,114",123 minutes,Sony Pictures Classics
8,14,"""Love Ranch"" is a bittersweet love story that ...",R,Drama,Taylor Hackford,Mark Jacobson,"Jun 30, 2010","Nov 9, 2010",$,"134,904",117 minutes,NaN
9,15,When a diamond expedition in the Congo is lost...,PG-13,Action and Adventure|Mystery and Suspense|Scie...,Frank Marshall,John Patrick Shanley,"Jun 9, 1995","Jul 27, 1999",NaN,NaN,108 minutes,NaN


In [4]:
movie_info.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1560 entries, 0 to 1559
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   id            1560 non-null   int64 
 1   synopsis      1498 non-null   object
 2   rating        1557 non-null   object
 3   genre         1552 non-null   object
 4   director      1361 non-null   object
 5   writer        1111 non-null   object
 6   theater_date  1201 non-null   object
 7   dvd_date      1201 non-null   object
 8   currency      340 non-null    object
 9   box_office    340 non-null    object
 10  runtime       1530 non-null   object
 11  studio        494 non-null    object
dtypes: int64(1), object(11)
memory usage: 146.4+ KB


In [39]:
movie_info['box_office'] = pd.to_numeric(movie_info['box_office'].str.replace(',', ''))

In [40]:
movie_info.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 340 entries, 1 to 1555
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   id            340 non-null    int64         
 1   synopsis      340 non-null    object        
 2   rating        340 non-null    object        
 3   genre         340 non-null    object        
 4   director      299 non-null    object        
 5   writer        273 non-null    object        
 6   theater_date  334 non-null    datetime64[ns]
 7   dvd_date      334 non-null    object        
 8   currency      340 non-null    object        
 9   box_office    340 non-null    int64         
 10  runtime       338 non-null    object        
 11  studio        305 non-null    object        
 12  day_of_week   334 non-null    object        
dtypes: datetime64[ns](1), int64(2), object(10)
memory usage: 47.2+ KB


In [43]:
movie_info.head()

,id,synopsis,rating,genre,director,writer,theater_date,dvd_date,currency,box_office,runtime,studio,day_of_week
1,3,"New York City, not-too-distant-future: Eric Pa...",R,Drama|Science Fiction and Fantasy,David Cronenberg,David Cronenberg|Don DeLillo,2012-08-17,"Jan 1, 2013",$,600000,108 minutes,Entertainment One,Friday
6,10,Some cast and crew from NBC's highly acclaimed...,PG-13,Comedy,Jake Kasdan,Mike White,2002-01-11,"Jun 18, 2002",$,41032915,82 minutes,Paramount Pictures,Friday
7,13,"Stewart Kane, an Irishman living in the Austra...",R,Drama,Ray Lawrence,Raymond Carver|Beatrix Christian,2006-04-27,"Oct 2, 2007",$,224114,123 minutes,Sony Pictures Classics,Thursday
8,14,"""Love Ranch"" is a bittersweet love story that ...",R,Drama,Taylor Hackford,Mark Jacobson,2010-06-30,"Nov 9, 2010",$,134904,117 minutes,NaN,Wednesday
15,22,Two-time Academy Award Winner Kevin Spacey giv...,R,Comedy|Drama|Mystery and Suspense,George Hickenlooper,Norman Snider,2010-12-17,"Apr 5, 2011",$,1039869,108 minutes,ATO Pictures,Friday


In [44]:
# turning the theater_date column into a datetime object
movie_info['theater_date'] = pd.to_datetime(movie_info['theater_date'], format='%b %d, %Y')

In [45]:
movie_info.head()

,id,synopsis,rating,genre,director,writer,theater_date,dvd_date,currency,box_office,runtime,studio,day_of_week
1,3,"New York City, not-too-distant-future: Eric Pa...",R,Drama|Science Fiction and Fantasy,David Cronenberg,David Cronenberg|Don DeLillo,2012-08-17,"Jan 1, 2013",$,600000,108 minutes,Entertainment One,Friday
6,10,Some cast and crew from NBC's highly acclaimed...,PG-13,Comedy,Jake Kasdan,Mike White,2002-01-11,"Jun 18, 2002",$,41032915,82 minutes,Paramount Pictures,Friday
7,13,"Stewart Kane, an Irishman living in the Austra...",R,Drama,Ray Lawrence,Raymond Carver|Beatrix Christian,2006-04-27,"Oct 2, 2007",$,224114,123 minutes,Sony Pictures Classics,Thursday
8,14,"""Love Ranch"" is a bittersweet love story that ...",R,Drama,Taylor Hackford,Mark Jacobson,2010-06-30,"Nov 9, 2010",$,134904,117 minutes,NaN,Wednesday
15,22,Two-time Academy Award Winner Kevin Spacey giv...,R,Comedy|Drama|Mystery and Suspense,George Hickenlooper,Norman Snider,2010-12-17,"Apr 5, 2011",$,1039869,108 minutes,ATO Pictures,Friday


In [46]:
# i confirm that the dates are a datetime object
movie_info.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 340 entries, 1 to 1555
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   id            340 non-null    int64         
 1   synopsis      340 non-null    object        
 2   rating        340 non-null    object        
 3   genre         340 non-null    object        
 4   director      299 non-null    object        
 5   writer        273 non-null    object        
 6   theater_date  334 non-null    datetime64[ns]
 7   dvd_date      334 non-null    object        
 8   currency      340 non-null    object        
 9   box_office    340 non-null    int64         
 10  runtime       338 non-null    object        
 11  studio        305 non-null    object        
 12  day_of_week   334 non-null    object        
dtypes: datetime64[ns](1), int64(2), object(10)
memory usage: 47.2+ KB


In [47]:
# now i can use the datetime methods to my advantage.
movie_info['theater_date'].dt.day_name()

1          Friday
6          Friday
7        Thursday
8       Wednesday
15         Friday
          ...    
1541       Friday
1542       Friday
1545       Friday
1546       Friday
1555       Friday
Name: theater_date, Length: 340, dtype: object

In [48]:
# now i can create another column so that i can refrence what day the movies came out.
# i can simply make a column with these days
movie_info['day_of_week'] = movie_info['theater_date'].dt.day_name()

In [49]:
movie_info.head()

,id,synopsis,rating,genre,director,writer,theater_date,dvd_date,currency,box_office,runtime,studio,day_of_week
1,3,"New York City, not-too-distant-future: Eric Pa...",R,Drama|Science Fiction and Fantasy,David Cronenberg,David Cronenberg|Don DeLillo,2012-08-17,"Jan 1, 2013",$,600000,108 minutes,Entertainment One,Friday
6,10,Some cast and crew from NBC's highly acclaimed...,PG-13,Comedy,Jake Kasdan,Mike White,2002-01-11,"Jun 18, 2002",$,41032915,82 minutes,Paramount Pictures,Friday
7,13,"Stewart Kane, an Irishman living in the Austra...",R,Drama,Ray Lawrence,Raymond Carver|Beatrix Christian,2006-04-27,"Oct 2, 2007",$,224114,123 minutes,Sony Pictures Classics,Thursday
8,14,"""Love Ranch"" is a bittersweet love story that ...",R,Drama,Taylor Hackford,Mark Jacobson,2010-06-30,"Nov 9, 2010",$,134904,117 minutes,NaN,Wednesday
15,22,Two-time Academy Award Winner Kevin Spacey giv...,R,Comedy|Drama|Mystery and Suspense,George Hickenlooper,Norman Snider,2010-12-17,"Apr 5, 2011",$,1039869,108 minutes,ATO Pictures,Friday


In [50]:
movie_info['day_of_week'].value_counts()

Friday       283
Wednesday     32
Thursday      10
Monday         4
Tuesday        3
Sunday         1
Saturday       1
Name: day_of_week, dtype: int64

In [51]:
# i want to view the earliest date in this data and max
movie_info['theater_date'].min()

Timestamp('1958-06-24 00:00:00')

In [52]:
movie_info['theater_date'].max()

Timestamp('2018-02-23 00:00:00')

In [53]:
# i can also subtract dates in order to view the time between those two dates
# this is called time delta
movie_info['theater_date'].max() - movie_info['theater_date'].min()

# pretty cool to see how many days are between 1921 and 2018!

Timedelta('21794 days 00:00:00')

In [54]:
movie_info = movie_info.dropna(subset=['box_office'])

In [55]:
movie_info['box_office'].isnull().sum()

0

In [56]:
new_subset = movie_info[['theater_date', 'box_office', 'day_of_week']]

In [57]:
new_subset.reset_index(drop=True, inplace=True)

In [58]:
new_subset.head(10)

,theater_date,box_office,day_of_week
0,2012-08-17,600000,Friday
1,2002-01-11,41032915,Friday
2,2006-04-27,224114,Thursday
3,2010-06-30,134904,Wednesday
4,2010-12-17,1039869,Friday
5,2013-12-20,99165609,Friday
6,2013-12-25,20518224,Wednesday
7,2004-05-14,1971135,Friday
8,2003-09-19,312136,Friday
9,2003-05-16,201010,Friday


In [59]:
new_subset.isnull().sum()

theater_date    6
box_office      0
day_of_week     6
dtype: int64

In [60]:
new_subset = new_subset.dropna(subset=['theater_date'])

In [61]:
new_subset = new_subset.dropna(subset=['day_of_week'])

In [124]:
new_subset.isnull().sum()

theater_date    0
box_office      0
day_of_week     0
dtype: int64

In [125]:
new_subset.groupby("day_of_week").agg({"box_office": ["mean", "median"]})

box_office            
                     mean      median
day_of_week                          
Friday       3.396972e+07  13900000.0
Monday       7.094144e+07  27788904.0
Saturday     2.839256e+06   2839256.0
Sunday       8.616662e+06   8616662.0
Thursday     8.488508e+07  79296714.5
Tuesday      4.527502e+07  33200000.0
Wednesday    6.178961e+07  19946849.0

In [63]:
final_subset = new_subset[['theater_date', 'box_office', 'day_of_week']]
final_subset.sort_values(['box_office', 'day_of_week'], inplace=True)

In [112]:
# up to this point i have made a tidy table. but in doing so i have created some redundancy.
# i can see that the day name appear in multiple rows.
final_subset.head(70)

,theater_date,box_office,day_of_week
205,2009-08-28,363,Friday
76,2010-07-30,2367,Friday
145,2013-11-22,3328,Friday
143,1974-12-25,8300,Wednesday
278,2011-08-26,8856,Friday
...,...,...,...
231,2010-04-09,1121000,Friday
264,2009-08-14,1149350,Friday
265,2004-09-24,1260219,Friday
85,2013-05-31,1300000,Friday


In [67]:
# a way to remove this redundancy is to create a separate table with days data and linking it to the box office using-
# an index. this is called data normalization.
days = final_subset[['day_of_week', 'box_office']].drop_duplicates()
days.head(15)

,day_of_week,box_office
205,Friday,363
76,Friday,2367
145,Friday,3328
143,Wednesday,8300
278,Friday,8856
298,Friday,16632
101,Friday,22099
137,Wednesday,25658
180,Friday,35966
127,Friday,39712


In [68]:
# since the index is unique i need to carry it explicitly as a column.
# i will reset the index and rename it id
days.index.name = 'id'
my_id = days.reset_index()
my_id.head(15)

,id,day_of_week,box_office
0,205,Friday,363
1,76,Friday,2367
2,145,Friday,3328
3,143,Wednesday,8300
4,278,Friday,8856
5,298,Friday,16632
6,101,Friday,22099
7,137,Wednesday,25658
8,180,Friday,35966
9,127,Friday,39712


In [69]:
# i will do a merge between the my_sid and the final dataframe
pd.merge(my_id, final_subset, on=['day_of_week', 'box_office']).head()

,id,day_of_week,box_office,theater_date
0,205,Friday,363,2009-08-28
1,76,Friday,2367,2010-07-30
2,145,Friday,3328,2013-11-22
3,143,Wednesday,8300,1974-12-25
4,278,Friday,8856,2011-08-26


In [70]:
# heres a simplified table
tidy = pd.merge(my_id, final_subset, on=['day_of_week', 'box_office']).drop(['theater_date'],axis=1)
tidy.head(15)

,id,day_of_week,box_office
0,205,Friday,363
1,76,Friday,2367
2,145,Friday,3328
3,143,Wednesday,8300
4,278,Friday,8856
5,298,Friday,16632
6,101,Friday,22099
7,137,Wednesday,25658
8,180,Friday,35966
9,127,Friday,39712


In [74]:
tidy.loc[tidy[tidy.day_of_week == 'Friday']['box_office'].idxmin()]

id                205
day_of_week    Friday
box_office        363
Name: 0, dtype: object

In [78]:
my_id.query('id == 100')

,id,day_of_week,box_office
282,100,Wednesday,88800000


In [84]:
final_subset.query('day_of_week == "Friday"')

,theater_date,box_office,day_of_week
205,2009-08-28,363,Friday
76,2010-07-30,2367,Friday
145,2013-11-22,3328,Friday
278,2011-08-26,8856,Friday
298,2011-02-18,16632,Friday
...,...,...,...
271,2003-05-02,214813155,Friday
106,2002-04-19,241250669,Friday
237,2002-11-15,261835892,Friday
254,2012-11-09,299300000,Friday


In [88]:
tidy.query('(box_office > 88800000) and (day_of_week in ["Friday"])')

,id,day_of_week,box_office
286,150,Friday,89602378
287,318,Friday,90672025
288,216,Friday,93008426
289,18,Friday,93300000
291,319,Friday,97661826
292,327,Friday,98000000
293,5,Friday,99165609
295,123,Friday,102981571
296,93,Friday,104880310
297,165,Friday,105765605


In [96]:
boxoffice_mean = tidy.box_office.describe().mean()

In [99]:
tidy.query('box_office  > @boxoffice_mean')

,id,day_of_week,box_office
267,229,Friday,66580191
268,202,Friday,67169549
269,99,Friday,67631157
270,26,Friday,67771442
271,192,Wednesday,71844424
...,...,...,...
329,237,Friday,261835892
330,142,Wednesday,279167575
331,254,Friday,299300000
332,181,Friday,303001229


In [109]:
monday = tidy.query('(box_office > @boxoffice_mean) and (day_of_week in ["Monday"])')
monday.describe()

,id,box_office
count,1.0,1.0
mean,317.0,227946274.0
std,NaN,NaN
min,317.0,227946274.0
25%,317.0,227946274.0
50%,317.0,227946274.0
75%,317.0,227946274.0
max,317.0,227946274.0


In [106]:
tuesday = tidy.query('(box_office > @boxoffice_mean) and (day_of_week in ["Tuesday"])')
tuesday.describe()

,id,box_office
count,1.0,1.0
mean,247.0,102515793.0
std,NaN,NaN
min,247.0,102515793.0
25%,247.0,102515793.0
50%,247.0,102515793.0
75%,247.0,102515793.0
max,247.0,102515793.0


In [107]:
wednesday = tidy.query('(box_office > @boxoffice_mean) and (day_of_week in ["Wednesday"])')
wednesday.describe()

,id,box_office
count,13.000000,1.300000e+01
mean,155.769231,1.395062e+08
std,102.470446,9.421097e+07
min,10.000000,7.184442e+07
25%,83.000000,8.115767e+07
50%,142.000000,9.511101e+07
75%,233.000000,1.320889e+08
max,302.000000,3.680000e+08


In [108]:
thursday = tidy.query('(box_office > @boxoffice_mean) and (day_of_week in ["Thursday"])')
thursday.describe()

,id,box_office
count,6.000000,6.000000e+00
mean,154.666667,1.328211e+08
std,55.952361,6.570193e+07
min,51.000000,7.560432e+07
25%,144.500000,9.191262e+07
50%,169.500000,1.230870e+08
75%,191.500000,1.327141e+08
max,203.000000,2.577041e+08


In [110]:
friday = tidy.query('(box_office > @boxoffice_mean) and (day_of_week in ["Friday"])')
friday.describe()

,id,box_office
count,46.000000,4.600000e+01
mean,188.282609,1.288604e+08
std,97.351520,5.974440e+07
min,5.000000,6.658019e+07
25%,100.750000,8.986979e+07
50%,215.000000,1.116149e+08
75%,256.250000,1.481347e+08
max,332.000000,3.030012e+08


In [115]:
saturday = tidy.query('(box_office > @boxoffice_mean) and (day_of_week in ["Saturday"])')
saturday.describe()

,id,box_office
count,0.0,0.0
mean,NaN,NaN
std,NaN,NaN
min,NaN,NaN
25%,NaN,NaN
50%,NaN,NaN
75%,NaN,NaN
max,NaN,NaN


In [119]:
tidy.query('day_of_week == "Saturday"')

,id,day_of_week,box_office
94,33,Saturday,2839256


In [120]:
sunday = tidy.query('(box_office > @boxoffice_mean) and (day_of_week in ["Sunday"])')
sunday.describe()

,id,box_office
count,0.0,0.0
mean,NaN,NaN
std,NaN,NaN
min,NaN,NaN
25%,NaN,NaN
50%,NaN,NaN
75%,NaN,NaN
max,NaN,NaN


In [121]:
tidy.query('day_of_week == "Sunday"')

,id,day_of_week,box_office
133,191,Sunday,8616662


In [123]:
tidy.groupby("day_of_week").agg({"box_office": ["mean", "median"]})

box_office            
                     mean      median
day_of_week                          
Friday       3.396972e+07  13900000.0
Monday       7.094144e+07  27788904.0
Saturday     2.839256e+06   2839256.0
Sunday       8.616662e+06   8616662.0
Thursday     8.488508e+07  79296714.5
Tuesday      4.527502e+07  33200000.0
Wednesday    6.178961e+07  19946849.0